In [ ]:
from pytz import timezone
from IPython.display import clear_output
from time import sleep

creates artificial return dataframe to use in notification. Dataframe
To be replaced with global variable of API pull with current data.

In [ ]:
def init():
    global artificial_return
    artificial_return = pd.DataFrame({"Artificail_Return": np.random.normal
                                      (loc = 0.1/252, scale = 0.2/(252**0.5), size = 100000)})
    dates = pd.date_range(start = ( datetime.datetime.today() - pd.DateOffset(seconds=99999)), 
                          end= datetime.datetime.today(), 
                          tz = "US/Pacific",
                         freq = "S")
    artificial_return["date"] = dates
    artificial_return.date = pd.to_datetime(artificial_return.date.dt.strftime("%Y-%m-%d %H:%M:%S"))

    artificial_return.set_index("date", inplace = True)

In [ ]:
def get_max_time(df):
    global max_date
    try:
        if isinstance(df, pd.DataFrame):
            max_date = df.iloc[-1].name
    except:
        return("Please check dates")
    max_date = pd.to_datetime(max_date).tz_localize('US/Eastern')
    print("Max time found in database", max_date)
    max_date = int(int(max_date.asm8)/1e6) # match date length of apis parameters which is 13 digits long
    max_date = int(max_date + 1e3) #skip 1 minute past last max date + 1e4*6(1min) (1e3 is one second)
def unix_to_date(unix):
    return pd.to_datetime(unix, unit = "ms").tz_localize('UTC').tz_convert('US/Eastern')
def date_to_unix(date):
    try:
        return int(int(pd.to_datetime(str(date)).tz_localize('US/Eastern').asm8)/1e6)
    except:
        return int(int(pd.to_datetime(str(date)).tz_convert('US/Eastern').asm8)/1e6)
def check_value(current_price, mean_price):
     if current_price > mean_price:
        return "Overvalued"
    elif current_price < mean_price:
        return "Undervalued"
    else:
        return "Price is at mean value."
def fetch_streams():
    global updates
    print(f"**************Fetching new price streams as last date found {str(unix_to_date(max_date))}**************")
    new_feed_times = pd.date_range(start = unix_to_date(max_date), 
                      end= datetime.datetime.now(timezone('US/Eastern')), 
                      tz = "US/Eastern",
                     freq = "S")
    updates = pd.DataFrame({"Artificail_Return": np.random.normal(loc = np.mean(artificial_return.values), 
                                                                  scale = np.std(artificial_return.values), size = len(new_feed_times))})
    updates["date"] = new_feed_times
    updates.date = pd.to_datetime(updates.date.dt.strftime("%Y-%m-%d %H:%M:%S"))

    updates.set_index("date", inplace = True)
def conditions(row):
    return [
        "background-color: red; color: white"
        if cell<= 0.0
        else  "background-color: green; color: white"
        for cell in row]

In [ ]:
import time, ssl, smtplib, os
import pandas as pd
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.header import Header

# DATA_PATH = "../models_DB.h5"

def send_email(check_trend):
    # with pd.HDFStore(DATA_PATH) as store:
    #     max_date = store['predictions/news/daily'].index.get_level_values(0).max().strftime('%Y-%m-%d')
    #     labels = store['predictions/news/daily'].loc[max_date]
    #     labels = labels[labels.BUY_V1 != labels.BUY_V2]
    #     labels.to_csv('%s_Predicitions.csv'% max_date)

    user = "firobeid92@gmail.com"
    key = "******"
    # students = list(pd.read_csv(r"C:\Users\feras.FIROMEGAPC\Desktop\Job Related\2U\Cohorts\Berkely\UCB-VIRT-FIN-PT-09-2022-U-B\students.csv").loc[:,"Emails"].dropna())
    to = ['feras.obeid@lau.edu'] #+ students

    subject = 'Updates for %s' % unix_to_date(max_date)
    email_body = """\
    Hello, 
    The stock is %s, an order will be submitted based on that!

    Yours Sincerely,
    Firas's Algo Streamer"""%check_trend

    # attachment = '%s_Predicitions.csv'% max_date
    ### Define email ###
    message = MIMEMultipart()
    message['From'] = Header(user.split("@")[0])
    # message['To'] = Header(to)     
    message['Subject'] = Header(subject)
    message.attach(MIMEText(email_body, 'plain', 'utf-8'))
    # att_name = os.path.basename(attachment)
    # att1 = MIMEText(open(attachment, 'rb').read(), 'base64', 'utf-8')
    # att1['Content-Type'] = 'application/octet-stream'
    # att1['Content-Disposition'] = 'attachment; filename=' + att_name
    # message.attach(att1)
    context = ssl.create_default_context()
    email_port = 465
    with smtplib.SMTP_SSL(host = "smtp.gmail.com", port = email_port, context = context) as server:
        server.login(user, key)
        print(server.ehlo())
        if server.ehlo()[0] == 250:
            server.sendmail(key,to, message.as_string())
            print('Email sent successfully!')
            # os.remove('%s_Predicitions.csv'% max_date)
            server.quit()
        else:
            print(f'Unable to establish connection with server! Error code: {server.ehlo()[0]}')
            server.quit()


In [ ]:
def run():
    global artificial_return
    init()
    get_max_time(artificial_return)

    while max_date < date_to_unix(datetime.datetime.now(timezone('US/Eastern')).strftime("%Y-%m-%d %H:%M:%S")) + 1e4*6: #add extra 1e4*6 1 min latency
        # Get most recent second data since init() till exactly now
        fetch_streams()
        # 
        display(updates.style.apply(conditions))
        #Get Last date from temp df for memory efficiny 
        get_max_time(updates)
        #Append to the original df to keep track of all returns 
        artificial_return = artificial_return.append(updates)
        # Calculate if current prices coming in are greater then previous one's
        print(round(np.mean(updates.values),6))
        print(round(np.mean(artificial_return.values),6))
        check_trend = check_value(round(np.mean(updates.values),6), round(np.mean(artificial_return.values),6))
        print(check_trend)
        sdf = Random(interval='200ms', freq='50ms')
        # clear_output(wait=True)
        sleep(1)
        # if check_trend == "Undervalued":
        #     send_email(check_trend)
        # elif check_trend == "Overvalued":
        #     send_email(check_trend)

In [ ]:
run()